In [1]:
import requests, json
import pandas as pd
from urllib.parse import quote

In [2]:
#API 키 가져오기
with open('C:/workspace/kakao.txt') as file:
    road_key = file.read()

In [3]:
# URL 만들기
base_url = 'https://dapi.kakao.com/v2/local/search/address.json'
addr = '제주특별자치도 제주시 남광북1길 1 제주법조타워 1,2층'
url = f'{base_url}?query={quote(addr)}'

In [4]:
#Header : Authorization: KakaoAK ${REST_API_KEY}
header = {'Authorization': f'KakaoAK {road_key}'}

In [5]:
#Kakao Local API 호출하여 결과 가져오기
result = requests.get(url, headers=header).json()
result

{'documents': [{'address': {'address_name': '제주특별자치도 제주시 이도이동 1065-6',
    'b_code': '5011010400',
    'h_code': '5011054000',
    'main_address_no': '1065',
    'mountain_yn': 'N',
    'region_1depth_name': '제주특별자치도',
    'region_2depth_name': '제주시',
    'region_3depth_h_name': '이도2동',
    'region_3depth_name': '이도이동',
    'sub_address_no': '6',
    'x': '126.534320623238',
    'y': '33.494593216571'},
   'address_name': '제주특별자치도 제주시 남광북1길 1',
   'address_type': 'ROAD_ADDR',
   'road_address': {'address_name': '제주특별자치도 제주시 남광북1길 1',
    'building_name': '제주법조타워',
    'main_building_no': '1',
    'region_1depth_name': '제주특별자치도',
    'region_2depth_name': '제주시',
    'region_3depth_name': '이도이동',
    'road_name': '남광북1길',
    'sub_building_no': '',
    'underground_yn': 'N',
    'x': '126.534320623238',
    'y': '33.494593216571',
    'zone_no': '63223'},
   'x': '126.534320623238',
   'y': '33.494593216571'}],
 'meta': {'is_end': True, 'pageable_count': 1, 'total_count': 1}}

In [6]:
lat = result['documents'][0]['y'] #위도(latitude)
lng = result['documents'][0]['x'] #경도(latitude)
dong=result['documents'][0]['address']['region_3depth_name']

In [7]:
lat, lng, dong

('33.494593216571', '126.534320623238', '이도이동')

In [9]:
#제주도 은행 정보 완성하기
df = pd.read_csv("./제주도 은행지점 및 ATM 목록.csv")
df

,은행명,주소
0,KB국민은행 제주종합금융센터,"제주특별자치도 제주시 남광북1길 1 제주법조타워 1,2층"
1,KB국민은행 연북로지점,제주특별자치도 제주시 연북로 50
2,하나은행 제주금융센터,제주특별자치도 제주시 중앙로 58
3,신한은행 제주지점,제주특별자치도 제주시 중앙로 166
4,하나은행 신제주지점,제주특별자치도 제주시 1100로 3308 1층
...,...,...
157,태흥새마을금고 제1분사무소,제주특별자치도 서귀포시 남원읍 태위로 810
158,스마트저축은행 제주여신전문출장소,제주특별자치도 제주시 중앙로 386 2층
159,삼호저축은행 제주지점,제주특별자치도 제주시 중앙로 535
160,한국투자저축은행 제주여신전문출장소,제주특별자치도 제주시 노형로 407 노형타워 4층


In [10]:
lat_list, lng_list, dong_list = [], [], [] #빈 리스트 생성. 위도와 경도 값 저장
for i in df.index: #df의 각 행에 대해 반복
    url = f'{base_url}?query={quote(df["주소"][i])}'
    result = requests.get(url, headers=header).json() #API에 요청을 보내고 응답을 JSON 형식으로 받음
    try:
        lat_list.append(result['documents'][0]['y']) #위도(y) 추출하여 lat_list에 추가
        lng_list.append(result['documents'][0]['x']) #경도(x) 추출하여 lng_list에 추가
        dong_list.append(result['documents'][0]['address']['region_3depth_name'])
    except:
        print(df.은행명[i])  
#try-except 구문: try블록 실행 중 예외 발생 시 except블록 실행

호남새마을금고 도남지점
제주은행 제주특별자치도지점


In [11]:
df[df.은행명.isin(['호남새마을금고 도남지점'])].주소

97     제주특별자치도 제주시 청사로 85 103,105호 
Name: 주소, dtype: object

In [12]:
#카카오맵에서 직접 검색해서 주소 수정
df.iloc[97]['주소'] = '제주특별자치도 제주시 청사로 85'
df.iloc[97]

은행명          호남새마을금고 도남지점
주소     제주특별자치도 제주시 청사로 85
Name: 97, dtype: object

In [13]:
df[df.은행명.isin(['제주은행 제주특별자치도지점'])].주소

122     제주특별자치도 제주시 문연로 6 제2별관 1층 
Name: 주소, dtype: object

In [14]:
#카카오맵에서 직접 검색해서 주소 수정
df.iloc[122]['주소'] = '제주특별자치도 제주시 문연로 6'
df.iloc[122]

은행명       제주은행 제주특별자치도지점
주소     제주특별자치도 제주시 문연로 6
Name: 122, dtype: object

In [15]:
#수정된 데이터로 다시 위도경도 받아오기
lat_list, lng_list, dong_list = [], [], [] #빈 리스트 생성. 위도와 경도 값 저장
for i in df.index: #df의 각 행에 대해 반복
    url = f'{base_url}?query={quote(df["주소"][i])}'
    result = requests.get(url, headers=header).json() #API에 요청을 보내고 응답을 JSON 형식으로 받음
    try:
        lat_list.append(result['documents'][0]['y']) #위도(y) 추출하여 lat_list에 추가
        lng_list.append(result['documents'][0]['x']) #경도(x) 추출하여 lng_list에 추가
        dong_list.append(result['documents'][0]['address']['region_3depth_name'])
    except:
        print(df.은행명[i])
#try-except 구문: try블록 실행 중 예외 발생 시 except블록 실행

In [16]:
df['위도']=lat_list
df['경도']=lng_list
df['주소2']=dong_list

In [17]:
#제주도 은행 위도 경도 동 리스트
lat_list
lng_list
dong_list

['이도이동',
 '노형동',
 '일도일동',
 '이도일동',
 '노형동',
 '일도일동',
 '노형동',
 '노형동',
 '서귀동',
 '서귀동',
 '강정동',
 '서귀동',
 '아라이동',
 '이도일동',
 '서호동',
 '연동',
 '삼도일동',
 '삼도일동',
 '이도일동',
 '이도이동',
 '노형동',
 '일도이동',
 '용담일동',
 '연동',
 '노형동',
 '이도이동',
 '노형동',
 '노형동',
 '서홍동',
 '강정동',
 '오라일동',
 '노형동',
 '서귀동',
 '노형동',
 '이도이동',
 '동홍동',
 '서귀동',
 '이도이동',
 '영평동',
 '연동',
 '아라일동',
 '이도이동',
 '이도이동',
 '일도이동',
 '노형동',
 '이도일동',
 '연동',
 '용담이동',
 '이도이동',
 '외도일동',
 '서귀동',
 '노형동',
 '연동',
 '한림읍 한림리',
 '도련일동',
 '연동',
 '아라일동',
 '도남동',
 '이도일동',
 '표선면 표선리',
 '아라일동',
 '도남동',
 '용담일동',
 '용담일동',
 '화북일동',
 '서귀동',
 '외도일동',
 '이도일동',
 '이도이동',
 '노형동',
 '호근동',
 '삼도이동',
 '동홍동',
 '일도일동',
 '대정읍 하모리',
 '노형동',
 '일도이동',
 '동홍동',
 '용담이동',
 '성산읍 고성리',
 '노형동',
 '서홍동',
 '연동',
 '이도이동',
 '서홍동',
 '애월읍 하귀2리',
 '노형동',
 '한림읍 한림리',
 '이도이동',
 '삼양이동',
 '삼도일동',
 '노형동',
 '동홍동',
 '삼도일동',
 '용담이동',
 '연동',
 '조천읍 조천리',
 '도남동',
 '중문동',
 '연동',
 '강정동',
 '노형동',
 '이도이동',
 '도련일동',
 '연동',
 '법환동',
 '도남동',
 '노형동',
 '연동',
 '아라이동',
 '일도일동',
 '서호동',
 '애월읍 납읍리',
 '도련일동',
 '한림읍 귀덕리',
 '애월읍

In [18]:
df.to_csv("제주도 은행지점 및 ATM 목록.csv", index=False)

In [19]:
pd.read_csv('./제주도 은행지점 및 ATM 목록.csv')

,은행명,주소,위도,경도,주소2
0,KB국민은행 제주종합금융센터,"제주특별자치도 제주시 남광북1길 1 제주법조타워 1,2층",33.494593,126.534321,이도이동
1,KB국민은행 연북로지점,제주특별자치도 제주시 연북로 50,33.481592,126.486661,노형동
2,하나은행 제주금융센터,제주특별자치도 제주시 중앙로 58,33.512922,126.524205,일도일동
3,신한은행 제주지점,제주특별자치도 제주시 중앙로 166,33.504033,126.526443,이도일동
4,하나은행 신제주지점,제주특별자치도 제주시 1100로 3308 1층,33.481632,126.481719,노형동
...,...,...,...,...,...
157,태흥새마을금고 제1분사무소,제주특별자치도 서귀포시 남원읍 태위로 810,33.282822,126.732440,남원읍 태흥리
158,스마트저축은행 제주여신전문출장소,제주특별자치도 제주시 중앙로 386 2층,33.487872,126.539872,이도이동
159,삼호저축은행 제주지점,제주특별자치도 제주시 중앙로 535,33.475712,126.545205,아라일동
160,한국투자저축은행 제주여신전문출장소,제주특별자치도 제주시 노형로 407 노형타워 4층,33.485821,126.478301,노형동
